# llm 기본틀 만들기
- OpenAI Key 로딩, ChatOpenAI() 객체 생성

In [16]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv('../.env', override=True)

OPENAIAPI = os.getenv('OPENAIAPI')
# print(api_key)

# 모델 객체 생성

In [17]:
# gpt model 객체 생성
# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=OPENAIAPI)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x122350e60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x122351790>, root_client=<openai.OpenAI object at 0x15406a780>, root_async_client=<openai.AsyncOpenAI object at 0x15350cbc0>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트 작성

- https://github.com/f/awesome-chatgpt-prompts
검색 : financial
```
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- "Can you tell us what future stock market looks like based upon current conditions ?".
```

In [19]:
system_prompt = """
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".
"""
user_prompt = """
    {company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.
    야래의 기본 정보, 재무제표를 참고해 마크다운 형식의 투자 보고서를 한글로 작성해 주세요.

    기본정보:
    {basic_info}

    재무제표:
    {financial_statement}
    """

# 프롬프트 객체 생성

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
prompt

ChatPromptTemplate(input_variables=['basic_info', 'company', 'financial_statement'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nWant assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['basic_info', 'company', 'financial_statement'], input_types={}, partial_variables={}, template='\n    {company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.\n    

# OutPut 파서 객체 생성

In [21]:
output_parser = StrOutputParser()
output_parser

StrOutputParser()

In [29]:
# from stock_info import Stock 인식이 안될 경우 아래 코드 실행
import sys
import os

# 상위 디렉터리(LANGCHAIN_PJT)를 경로에 추가
sys.path.append(os.path.abspath(".."))


In [30]:
# chain 구성 : LCEL(LangChain Expression Language)
chain = prompt | llm | output_parser

# 회사 이름
# company = "MicroSoft"
# symbol = "MSFT"  # stock의 symbol 정보

company = "Apple Inc"
symbol = "AAPL"
# 기본정보 :  basic_info
# 재무제표: finacial_statement

# stock 정보를 객체로 모듈화해서 불러오기
from stock_info import Stock

stock = Stock(symbol)
req_value = {
    "company":company,
    # 기본정보 :  basic_info
    "basic_info": stock.get_basic_info(),
    # 재무제표: finacial_statement
    "financial_statement" : stock.get_financial_statement()
            }

response = chain.invoke(req_value)

In [31]:
print(response)

# Apple Inc. 투자보고서

## 기본정보
- 기업명: Apple Inc.
- 시가총액: 2,968,938,348,544
- 산업: 소비자 전자제품
- 섹터: 기술
- 정규직 종업원 수: 164,000 명
- 현재 주가: $198.78
- 기업 가치: 3,076,731,961,344

## 재무제표

### 손익계산서
| 항목              | 2024년  | 2023년  | 2022년  | 2021년  |
|:------------------|--------:|--------:|--------:|--------:|
| 총 수익           | 391,035억 | 383,285억 | 394,328억 | 365,817억 |
| 총 이익           | 180,683억 | 169,148억 | 170,782억 | 152,836억 |
| 영업 이익         | 123,216억 | 114,301억 | 119,437억 | 108,949억 |
| 순 이익           | 93,736억  | 96,995억  | 99,803억  | 94,680억  |

### 대차대조표
| 항목                                       | 2024년  | 2023년  | 2022년  | 2021년  |
|:------------------------------------------|--------:|--------:|--------:|--------:|
| 총 자산                                   | 364,980억 | 352,583억 | 352,755억 | 351,002억 |
| 총 부채 및 소수주의 이익 차지 지분 | 308,030억 | 290,437억 | 302,083억 | 287,912억 |
| 주주자 자본                             | 56,950억  | 62,146억  | 50,672억  | 63,090억  |

### 현금 흐름표
| 항목               